# Titanic competition with TensorFlow Decision Forests

This notebook will take you through the steps needed to train a baseline Gradient Boosted Trees Model using TensorFlow Decision Forests and creating a submission on the Titanic competition. 

TensorFlow Decision Forests是什么：tensorflow是由google开源的AI框架，而TF-DF是它专门用来处理表格数据的分支。代码里写的是python API，但底层核心计算引擎是C++

GBT：基础单元是二叉树。通过梯度来进行提升。

关于这棵二叉树：rootnode判断性别，第二层节点判断年龄，第三层节点判断船票等级……最终leafnode给出综合性别，年龄，船票后的预测得分。

假设第一棵树预测结果是0.3，但实际情况是1.0（存货），那么就产生了0.7的负梯度。第二棵树的终极目标，不再是预测这名乘客是死是活，而是预测那0.7的误差。

This notebook shows:

1. How to do some basic pre-processing. For example, the passenger names will be tokenized, and ticket names will be splitted in parts.
1. How to train a Gradient Boosted Trees (GBT) with default parameters
1. How to train a GBT with improved default parameters
1. How to tune the parameters of a GBTs
1. How to train and ensemble many GBTs

# Imports dependencies

In [ ]:
import numpy as np
#相当于math.h
import pandas as pd
#表格处理神器，底层把每一列数据当成高级的结构体数组
import os
#kaggle跟后台的Linux系统打交道，可以直接访问底层的硬盘路径

import tensorflow as tf
import tensorflow_decision_forests as tfdf

print(f"Found TF-DF {tfdf.__version__}")

# Load dataset

In [ ]:
#df就是dataframe（数据框），类似一个高级的二维结构体数组
train_df = pd.read_csv("/kaggle/input/titanic/train.csv")
serving_df = pd.read_csv("/kaggle/input/titanic/test.csv")
#将train和test分开处理

train_df.head(10)
#jupyter的每一个cell里，放在最后一行的变量或表达式的返回值会被系统自动捕获并打印

# Prepare dataset

We will apply the following transformations on the dataset.

1. Tokenize the names. For example, "Braund, Mr. Owen Harris" will become ["Braund", "Mr.", "Owen", "Harris"].
2. Extract any prefix in the ticket. For example ticket "STON/O2. 3101282" will become "STON/O2." and 3101282.

预处理：用python切割字符串

1.df = df.copy()这一步极其关键。查了资料才反应过来，python传表格进函数默认传的是引用（类似c语言中的指针）。如果不copy一份，在函数里乱改就会把外面的原数据彻底破坏掉。

2.x.split("")处理字符串的神器，完美代替c语言里的strtok

3..apply()就像一个高速for循环，把清洗函数挨个作用到表格的每一行上。

In [ ]:
def preprocess(df):
    df = df.copy()
    #类似malloc开辟新空间，防止污染原指针数据
    
    def normalize_name(x):
        #替代c语言中的strcat，直接去标点并重新拼接
        return " ".join([v.strip(",()[].\"'") for v in x.split(" ")])
        #"".join（数组）可以直接以空格为连接符，瞬间将所有碎词粘合成一个完整的字符串
    
    def ticket_number(x):
        return x.split(" ")[-1]
        #[-1]可以直接取数组中的最后一个元素（python的语法糖）
        
    def ticket_item(x):
        items = x.split(" ")
        if len(items) == 1:
            return "NONE"
        return "_".join(items[0:-1])
        #[0:-1]是切片语法，相当于从头取到倒数第二个
    #因为船票最后的数字大概率只是随机流水号，毫无预测价值
    
    df["Name"] = df["Name"].apply(normalize_name)
    df["Ticket_number"] = df["Ticket"].apply(ticket_number)
    df["Ticket_item"] = df["Ticket"].apply(ticket_item) 
    #apply相当于直接在底层写好了for循环
    return df
    
preprocessed_train_df = preprocess(train_df)
preprocessed_serving_df = preprocess(serving_df)

preprocessed_train_df.head(5)

Let's keep the list of the input features of the model. Notably, we don't want to train our model on the "PassengerId" and "Ticket" features.

挑出有用的线索（特征工程）

这一步的逻辑很简单：把没用的垃圾数据扔掉

比如passngerid只是个流水号，与存活率没有任何关系，如果喂给模型，可能会导致模型死记硬背“几号死了几号活了”导致过拟合

最重要的防作弊：必须把survived（存货状态）从特征里踢出去！因为这是我们要预测的答案。如果把答案混在线索里喂给AI，这就是典型的data leakage。

In [ ]:
input_features = list(preprocessed_train_df.columns)
#list是更高级的malloc动态数组，直接将表格上所有的列名拷贝出来，存在一个可以随时增删的字符串线性表中
input_features.remove("Ticket")
#ticket的价值在preprocess中已经被榨干
input_features.remove("PassengerId")
#passengerid是无价值信息
input_features.remove("Survived")
#survived是我们要预测的目标值Y，不能把它当特征X留在这个数组里喂给模型，否则就是把考试答案写在了题干上
#input_features.remove("Ticket_number")

print(f"Input features: {input_features}")

# Convert Pandas dataset to TensorFlow Dataset

组装数据流水线&简单的NLP切词

pandas的表格虽然直观，但底层不适合给tensorflow的底层c++引擎做超高速运算。这里用API把数据强制打包成了dataset格式

另外，tokenize_names这一步很有意思。它把名字切成了单独的单词（token）。这样模型在建树的时候，就能自己发现带有"Mr."和"Miss"的人的存活率是否有明显区别了。

In [ ]:
def tokenize_names(features, labels=None):
    #None：可以满足传入参数和不传入参数两种情况
    #features：自变量，labels：因变量
    #机器学习的本质：把大量的features和已知的labels喂给电脑，让电脑自己去把中间那个极其复杂的方程式给推导出来
    """Divite the names into tokens. TF-DF can consume text tokens natively."""
    features["Name"] =  tf.strings.split(features["Name"])
    return features, labels
    #return很巧妙：如果是train，就返回切好的题和答案；如果是test，就返回切好的题

train_ds = tfdf.keras.pd_dataframe_to_tf_dataset(preprocessed_train_df,label="Survived").map(tokenize_names)
serving_ds = tfdf.keras.pd_dataframe_to_tf_dataset(preprocessed_serving_df).map(tokenize_names)

# Train model with default parameters

### Train model

First, we are training a GradientBoostedTreesModel model with the default parameters.

第一次炼丹：建立baseline

在正式调参前，先用 Google 出厂的默认参数跑一版梯度提升树（Gradient Boosted Trees）。

关于GBT我的数学直觉：刚开始觉得机器学习很玄乎，但联系到最近学的高数就通透了。这个模型不是一次性长成一棵完美的树，而是后面长出的每一棵新树，都在去拟合上一棵树造成的误差（负梯度）。这不就是微积分里，利用偏导数寻找函数值下降最快方向的原理吗？每次顺着梯度下山走一小步，最后走到 Loss 误差最小的谷底。

In [ ]:
model = tfdf.keras.GradientBoostedTreesModel(
    #减少不必要的控制台打印
    verbose=0, # Very few logs
    features=[tfdf.keras.FeatureUsage(name=n) for n in input_features],
    exclude_non_specified_features=True, # Only use the features in "features"
    #featureusage配合这一行相当于告诉底层只能用提供的inputfeatures，其他数据一律不准碰
    random_seed=1234,
    #固定种子是为了保证实验“可复现”，从而调整其他参数引起的准确率变化一定是由参数引起的
)
model.fit(train_ds)

self_evaluation = model.make_inspector().evaluation()
print(f"Accuracy: {self_evaluation.accuracy} Loss:{self_evaluation.loss}")

# Train model with improved default parameters

Now you'll use some specific parameters when creating the GBT model

第二次炼丹：解析几何与高等数学

这里代码里有大量的#注释，明显是作者在电脑前反复修改数字、手动试错的痕迹。我也仔细研究了这些“底层齿轮”：

min_examples: 数据结构里讲二叉树，这就像是树向下递归分裂的 base case（基准条件）

SPARSE_OBLIQUE (倾斜分割): 普通的决策树只能画平行于坐标轴的线去切分数据。开启这个选项后，模型会在多维空间里做向量运算，寻找一个倾斜的超平面（Hyperplane）来切分

shrinkage (学习率): 相结合微积分，这就是模型顺着 Loss 函数负梯度“下山”时的步长（Step Size）。步子设小一点（比如 0.05），下山更稳，不容易直接跨过最低点

In [ ]:
model = tfdf.keras.GradientBoostedTreesModel(
    verbose=0, # Very few logs
    features=[tfdf.keras.FeatureUsage(name=n) for n in input_features],
    exclude_non_specified_features=True, # Only use the features in "features"
    
    #num_trees=2000,
    
    # Only for GBT.
    # A bit slower, but great to understand the model.
    # compute_permutation_variable_importance=True,
    
    # Change the default hyper-parameters
    # hyperparameter_template="benchmark_rank1@v1",
    
    #num_trees=1000,
    #tuner=tuner
    
    min_examples=1,
    #递归结束条件：只要节点里还剩一个样本就继续分裂，极限挖掘数据
    #对于树太深导致过拟合的问题，后面用别的参数去控制
    categorical_algorithm="RANDOM",

    shrinkage=0.05,
    #每一棵树对新残差只贡献0.05，当时越过Loss函数的极小值点

    split_axis="SPARSE_OBLIQUE",
    sparse_oblique_normalization="MIN_MAX",
    sparse_oblique_num_projections_exponent=2.0,
    #超平面分割：将分类问题转化为线性代数运算
    #传统的决策树只能水平切或者垂直切，而sparse_oblique允许在特征空间中画出“斜着的切面”
    #比如他不再问“票价是否大于100，而是问票价与仓位等级的加权组合是否超过某个阈值“
    #模型可以通过斜着切的能力用更短的树捕获更复杂的特征关系
    
    num_trees=2000,
    #validation_ratio=0.0,
    random_seed=1234,
    
)
model.fit(train_ds)

self_evaluation = model.make_inspector().evaluation()
print(f"Accuracy: {self_evaluation.accuracy} Loss:{self_evaluation.loss}")

Let's look at the model and you can also notice the information about variable importance that the model figured out

In [ ]:
model.summary()

# Make predictions

交卷：输出预测结果

终于到了最后一步。模型预测出来的并不是直接的0和1，而是“存活的概率”。模型遍历serving_ds的过程，本质上是让每个乘客在2000棵树的结构体数组里跑一遍指针遍历，最后将叶子节点的survival_score累加

所以这里设置了一个threshold=0.5的及格线。大于0.5的判定为 true，然后用.astype(int)强转成整型的1（这里就和 C 语言的 (int)强转一模一样）

最后用to_csv把内存里的结果存成硬盘上的答题卡文件，关掉index行号

In [ ]:
def prediction_to_kaggle_format(model, threshold=0.5):
    proba_survive = model.predict(serving_ds, verbose=0)[:,0]
    return pd.DataFrame({
        "PassengerId": serving_df["PassengerId"],
        #保持原有的乘客ID，方便kaggle机器对号入座
        
        "Survived": (proba_survive >= threshold).astype(int)
    })

def make_submission(kaggle_predictions):
    path="/kaggle/working/submission.csv"
    kaggle_predictions.to_csv(path, index=False)
    print(f"Submission exported to {path}")
    
kaggle_predictions = prediction_to_kaggle_format(model)
make_submission(kaggle_predictions)
!head /kaggle/working/submission.csv

# Training a model with hyperparameter tunning

Hyper-parameter tuning is enabled by specifying the tuner constructor argument of the model. The tuner object contains all the configuration of the tuner (search space, optimizer, trial and objective).


第三次炼丹：直接用算力来穷举

刚才看了作者手动改参数，我还在想 AI 工程师难道是个体力活？直到看到这个RandomSearch模块我才懂了

用 C 语言的眼光看，这其实就是在模型外面套了一个极度优化的、执行 1000 次的超级 for 循环（num_trials=1000）。通过 tuner.choice 传进去一个个数组，让机器自己在几百种组合里抛骰子试错。这让我明白了，在真实的工程中，比起死磕某一个数学公式，如何利用机器庞大的算力去自动化寻找最优解（全局最优节点 vs 局部最大深度），才是真正的降维打击。

In [ ]:
tuner = tfdf.tuner.RandomSearch(num_trials=1000)
#开启1000次独立实验的随即搜索

tuner.choice("min_examples", [2, 5, 7, 10])
tuner.choice("categorical_algorithm", ["CART", "RANDOM"])
#在这些候选值里掷骰子

local_search_space = tuner.choice("growing_strategy", ["LOCAL"])
local_search_space.choice("max_depth", [3, 4, 5, 6, 8])
#策略1：传统的”按层生长“，限制二叉树的最大深度

global_search_space = tuner.choice("growing_strategy", ["BEST_FIRST_GLOBAL"], merge=True)
global_search_space.choice("max_num_nodes", [16, 32, 64, 128, 256])
#策略2：现代的“按质生长”，限制二叉树的总结点数
#谁的效果好谁就先分叉，虽然这样生成的树往往不对称，但是更高效

#tuner.choice("use_hessian_gain", [True, False])
tuner.choice("shrinkage", [0.02, 0.05, 0.10, 0.15])
tuner.choice("num_candidate_attributes_ratio", [0.2, 0.5, 0.9, 1.0])


tuner.choice("split_axis", ["AXIS_ALIGNED"])
oblique_space = tuner.choice("split_axis", ["SPARSE_OBLIQUE"], merge=True)
oblique_space.choice("sparse_oblique_normalization",
                     ["NONE", "STANDARD_DEVIATION", "MIN_MAX"])
oblique_space.choice("sparse_oblique_weights", ["BINARY", "CONTINUOUS"])
oblique_space.choice("sparse_oblique_num_projections_exponent", [1.0, 1.5])
#针对倾斜分割的一系列细则开关

# Tune the model. Notice the `tuner=tuner`.
tuned_model = tfdf.keras.GradientBoostedTreesModel(tuner=tuner)
tuned_model.fit(train_ds, verbose=0)
#将“自动调参机器人”挂载到模型上
#tuned_model在fit时，不再只炼丹一次，而是疯狂的跑1000遍不同的参数组合

tuned_self_evaluation = tuned_model.make_inspector().evaluation()
print(f"Accuracy: {tuned_self_evaluation.accuracy} Loss:{tuned_self_evaluation.loss}")
#提取这1000次实验中胜出的参数组合

In the last line in the cell above, you can see the accuracy is higher than previously with default parameters and parameters set by hand.

This is the main idea behing hyperparameter tuning.

For more information you can follow this tutorial: [Automated hyper-parameter tuning](https://www.tensorflow.org/decision_forests/tutorials/automatic_tuning_colab)

# Making an ensemble

Here you'll create 100 models with different seeds and combine their results

This approach removes a little bit the random aspects related to creating ML models

In the GBT creation is used the `honest` parameter. It will use different training examples to infer the structure and the leaf values. This regularization technique trades examples for bias estimates.

颠覆我对机器学习认知的部分：我原本以为，拿高分就是要找出一个极其完美的“超级模型”。但原来工业界的杀手锏是“打群架”

Ensemble（模型集成）的底层逻辑：
作者不仅用了刚才的GBT，可能还加了随机森林random forest等其他不同算法训练出来的模型。然后，把这几个模型对同一个乘客预测的存活概率，做了一个简单的加权平均：$P_{final} = \frac{1}{N}\sum P_i$

在 C 语言里，这不过是一个极其简单的 for 循环累加求平均值的操作。但从概率论的角度来看，每个模型都有自己的“偏见”（方差），但只要它们犯错的方向不一样，求平均值就能极大程度地抵消误差，让最终的预测结果变得无比稳定。所谓“三个臭皮匠顶个诸葛亮”，大概就是这个数学原理吧

In [ ]:
predictions = None
num_predictions = 0
#类似c语言中初始化一个指向float数组的指针

for i in range(100):
    print(f"i:{i}")
    # Possible models: GradientBoostedTreesModel or RandomForestModel
    model = tfdf.keras.GradientBoostedTreesModel(
        verbose=0, # Very few logs
        features=[tfdf.keras.FeatureUsage(name=n) for n in input_features],
        exclude_non_specified_features=True, # Only use the features in "features"
        #每次循环都在底层重新实例化一个c++引擎结构体

        #min_examples=1,
        #categorical_algorithm="RANDOM",
        ##max_depth=4,
        #shrinkage=0.05,
        ##num_candidate_attributes_ratio=0.2,
        #split_axis="SPARSE_OBLIQUE",
        #sparse_oblique_normalization="MIN_MAX",
        #sparse_oblique_num_projections_exponent=2.0,
        #num_trees=2000,
        ##validation_ratio=0.0,
        random_seed=i,
        #多样性消除误差
        #前面固定种子是为了调参，这里随机种子是为了集成
        honest=True,
    )
    model.fit(train_ds)
    
    sub_predictions = model.predict(serving_ds, verbose=0)[:,0]
    if predictions is None:
        predictions = sub_predictions
    else:
        predictions += sub_predictions
    num_predictions += 1

predictions/=num_predictions

kaggle_predictions = pd.DataFrame({
        "PassengerId": serving_df["PassengerId"],
        "Survived": (predictions >= 0.5).astype(int)
    })

make_submission(kaggle_predictions)

# What is next

If you want to learn more about TensorFlow Decision Forests and its advanced features, you can follow the official documentation [here](https://www.tensorflow.org/decision_forests) 